# IN4080: obligatory assignment 4
 
The final mandatory assignment for IN4080 consists of two parts. The first is about the development of dialogue systems, and the second about machine translation.
You are required to get at least 12/20 points to pass. 

- We assume that you have read and are familiar with IFI’s requirements and guidelines for mandatory assignments, see [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-mandatory.html) and [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-guidelines.html).
- This is an individual assignment. You should not deliver joint submissions. 
- You may redeliver in Devilry before the deadline (__Sunday, November 3 at 23:59__).
- Only the last delivery will be read! If you deliver more than one file, put them into a zip-archive. You don't have to include in your delivery the data files already provided for this assignment. 
- Name your submission _your\_username\_in4080\_mandatory\_4_

Part 1 should be done on your local computer, as it relies on a speech interface that will not work on remote machines. For Part 2, using _Fox_ is preferable, at least for the fine-tuning task.

You should deliver a completed version of this Jupyter notebook, containing both your code and explanations about the steps you followed. We want to stress that simply submitting code is __not__ by itself sufficient to complete the assignment - we expect the notebook to also contain explanations of what you have implemented, along with motivations for the choices you made along the way. Preferably use whole sentences, and mathematical formulas if necessary. Explaining in your own words (using concepts we have covered through in the lectures) what you have implemented and reflecting on your solution is an important part of the learning process - take it seriously!

Regarding the use of LLMs (ChatGPT or similar): you are allowed to use them as 'sparring partner', for instance to clarify something you have not understood. However, you are __not__ allowed to use them to generate solutions (either in part or in full) to the assignment tasks. 


## Part 1 : Dialogue systems

Our objective in this part is to build a spoken conversational interface for a (simulated) elevator. 

### Basic setup

First, let's make sure that we have all the necessary Python modules:

In [ ]:
# !pip3 install ipywidgets pyaudio openai-whisper pyttsx3 setfit spacy jellyfish
# !python3 -m spacy download en_core_web_sm

The code for the simulated elevator is provided below. The elevator is displayed using simple widgets (where the current floor is shown in green). 

In [ ]:
import ipywidgets as widgets
from IPython.display import display

import time, random, string, threading
from typing import List, Tuple, Dict, Set

class BasicElevator:
    """Elevator simulated using a GUI"""
    
    def __init__(self, start_floor:int =1, nb_floors=10):
        """Initialised a new elevator, placed on the first floor"""
        
        # Current floor of the elevator
        self.cur_floor: int = start_floor

        # (Possibly empty) list of next floor stops to reach 
        self.next_stops : List[int] = []
        
        # Building the basic GUI showing the elevator
        display(self._build_gui(nb_floors))

        # Starts the thread executing the movements
        thread = threading.Thread(target=self.elevator_move_thread)
        thread.start()     
            
    def move_to_floor(self, floor_number : int):
        """Move to a given floor (by adding it to a stack of floors to reach)"""
        
        if floor_number < 1 or floor_number > len(self.floors):
            raise RuntimeError("Floor number must be between 1 and %i"%len(self.floors))

        self.next_stops.append(floor_number)
        
        
    def stop(self):
        """Stops all movements of the elevator"""
        self.next_stops.clear()

    def _build_gui(self, nb_floors):
        """Creates the GUI for the elevator, with a status label and a visual representation
        of the floors, where the current floor is indicated in green."""

        # Displaying the current status of the elevator (still or going up or down)
        status_label = widgets.HTML("<b>Status</b>: ")
        self.status = widgets.Label("Still")
        status_box = widgets.HBox([status_label, self.status])

        # Displaying the floors on a vertical axis
        self.floors = []
        floor_layout = widgets.Layout(width='50px', height='30px', border='2px solid black',justify_content="center")
        for i in range(1, nb_floors+1):
            floor = widgets.Label(value=str(i), layout=floor_layout)
            floor.style = {"background":("white" if i!=self.cur_floor else "lightgreen")}
            self.floors.append(floor)

        # Create a vertical box container to hold the boxes
        vbox = widgets.VBox([status_box] + self.floors[::-1])
        return vbox
    

    def elevator_move_thread(self, speed=1.0, latency=0.1):
        """Trigger a movement of the elevator if the list of next stops is not 
        empty. The movement continues until all goals are reached."""

        while True:
            while self.next_stops:
                if self.cur_floor == self.next_stops[0]:
                    del self.next_stops[0]
                    continue
                if self.cur_floor < self.next_stops[0]:
                    next_floor = self.cur_floor+1
                    self.status.value = "UP"
                elif self.cur_floor > self.next_stops[0]:
                    next_floor = self.cur_floor-1
                    self.status.value = "DOWN"
                time.sleep(speed)   
                self.floors[self.cur_floor-1].style.background = "white"
                self.floors[next_floor-1].style.background = "lightgreen"
                self.cur_floor = next_floor
            self.status.value = "Still"
            
            # Wait loop (until we have a goal in self.next_stops)
            time.sleep(latency)

The elevator can be easily controlled through the functions `move_to_floor` and `stop`:

In [ ]:
elevator = BasicElevator()
elevator.move_to_floor(6)

We will now make our elevator controllable through a speech interface instead of using function calls.

## Speech interface

First, make sure that you have installed `pyaudio` (for audio processing), `whisper` (for speech recognition), and `pyttsx3` (for speech synthesis).

The `TalkingElevator` class below extends the basic simulated elevator with speech input and output. 

Upon clicking on the recording button, speech is recorded from the user's microphone, and continues until the stop button is clicked. The speech recognition engine `Whisper` from OpenAI is then employed to transcribe the spoken input (either on GPU, if you have a GPU on your machine, or on CPU). The transcription result is then sent to the `process_input` function, which is responsible for determining the system response. 

We are going to focus on implementing this `process_input` method. Note this system reaction to new user inputs may comprise both verbal responses (to be uttered by the system through the `_say_to_user` method) and physical actions (through the `move_to_floor` and `stop` methods).

In [ ]:
# !pip install pyttsx3 pyaudio openai-whisper
# !pip install pyttsx3 speechrecognition openai-whisper

In [30]:
import threading, time
import numpy as np
from typing import List, Tuple, Dict, Set
import whisper, pyttsx3#, pyaudio, 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

class TalkingElevator(BasicElevator):
    """Extension of the simulated elevator with a speech interface"""
    
    def __init__(self):
        print("Loading TTS and ASR models", end="...", flush=True)
        self.tts_engine = pyttsx3.init()  
        self.asr_engine = whisper.load_model("small.en")
        print("Done")
        
        # Initializing the GUI
        BasicElevator.__init__(self)

        # Starts the dialogue
        self.dialogue_history = []
        self._say_to_user("Hi, what can I do for you today?")
    

    def process_input(self, user_input: str, conf_score:float=1.0):
        """Processes the (transcribed) user input, and respond appropriately 
        (through a verbal response and possibly also an action, such as moving floors)"""

        self._add_to_dialogue_history(user_input, speaker="user", conf_score=conf_score)

        # Dummy response. Should be replaced by the actual dialogue behaviour
        if "floor" in user_input.lower():
            floor_number = self._extract_floor_number(user_input)
        if floor_number:
            self._say_to_user(f"Okay, moving to floor {floor_number}.")
            self.move_to_floor(floor_number)
        else:
            self._say_to_user("Sorry, I didn't catch the floor number.")

    
    def _say_to_user(self, system_response: str):
        """Say something back to the user, and add the dialogue turn to the history. The 
        synthesis is done using the pyttsx3 library."""

        self._add_to_dialogue_history(system_response, speaker="elevator")
        
        # Stopping current TTS if one is active
        try:
            self.tts_engine.endLoop()
        except:
            pass
        self.tts_engine.say(system_response)
        self.tts_engine.runAndWait()


    def _add_to_dialogue_history(self, turn:str , speaker:str, conf_score:float=1.0):
        """Adds a new (user or system) turn to the dialogue history list, and displays it
            on the chat window displaying the turns"""

        self.dialogue_history.append({"speaker":speaker, "text":turn, 
                                        "conf_score":conf_score, "timesamp":time.time()})
        
        self.history_area.value += "&nbsp;<strong>%s</strong>:  %s"%(speaker.title(), turn)
        if conf_score < 1.0:
            self.history_area.value += " (%.2f)"%(conf_score)
        self.history_area.value += "<br>"

    def _build_gui(self, nb_floors):
        """GUI for the Talking elevator, comprising (beyond the simulated elevator from 
        BasicElevator) a chat window showing the dialogue turns, and buttons to record
        the user input. 
        The user should first click on the record button, then on stop when they have finished.
        Once the stop button is clicked, the audio is transcribed by Whisper, and finally 
        forwarded to the process_input function."""

        core_gui = BasicElevator._build_gui(self, nb_floors)

        self.frames = []
        self.recording = False

        def record(chunk_size=1024):
            """Record audio chunks to a buffer."""
            self.recording = True
            p = pyaudio.PyAudio()
            stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, 
                            frames_per_buffer=chunk_size)
            while self.recording:
                self.frames.append(stream.read(chunk_size))
            stream.close()

        def on_record_button_clicked(b):
            "Starts the recording"
            record_button.disabled=True
            stop_button.disabled=False
            self.frames = []  # Clear previous recordings
            thread = threading.Thread(target=record)
            thread.start()

        def on_stop_button_clicked(b):
            "stops the recording, runs Whisper, and forward the result to process_input"
            self.recording = False
            record_button.disabled=False
            stop_button.disabled=True
            audio_data = np.frombuffer(b"".join(self.frames), np.int16).astype(np.float32) * (1 / 32768.0)
            output = self.asr_engine.transcribe(audio_data)

            # We define the confidence score based on the log-probabilities
            conf_score = np.exp(np.mean([segment["avg_logprob"] for segment in output["segments"]]))
            # (and we push those up a bit, as the Whisper scores seem too low)
            conf_score = min(1, conf_score*1.2)

            # Finally, we process the input
            self.process_input(output["text"], conf_score)

        # The record and stop buttons
        record_button = widgets.Button(icon="microphone")
        stop_button = widgets.Button(icon="stop", disabled=True)
        record_button.on_click(on_record_button_clicked)
        stop_button.on_click(on_stop_button_clicked)
        
        # The chat area
        self.history_area = widgets.HTML(layout=widgets.Layout(width="600px", height="300px", 
                                                               border='1px solid black', overflow='scroll'))

        # The right side of the GUI
        right_side = widgets.VBox([widgets.Label(""), self.history_area, widgets.HBox([record_button, stop_button])])
        
        extended_gui = widgets.HBox([core_gui, right_side])
        return extended_gui


Let's give it a try:

In [ ]:
elevator = TalkingElevator()

In [ ]:
# import certifi
# import ssl
# print(certifi.where())

In [ ]:
# import os

# os.environ["SSL_CERT_FILE"] = "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/certifi/cacert.pem"

We hope that the speech recognition and speech synthesis will work correctly -- if it isn't the case, do let us know ! (audio processing in Python can be quite tricky and will work differently from OS to OS). [^1]

[^1]: If you are running on Linux and the TTS is not working, install the following packages on your machine: `sudo apt update && sudo apt install espeak ffmpeg libespeak1`

**Note**: The current implementation reloads the TTS and ASR models every time, which means that you may run into a "CUDA: out of memory" error if you reinitialise the `TalkingElevator` many times. If this happens, simply restart the Python kernel, which will clear the memory on both the CPU and the GPU. 


### Intent recognition

We wish our talking elevator to support the following functionalities:
 
- If the user express a wish to go to floor $X$ (where $X$ is an integer value between 1 and 10), the elevator should go to that floor. The interface should allow for several ways to express a given intent, such as "_Please go to the $X$-th floor_" or "_Floor $X$, please_".
- The user requests can also be relative, for instance "_Go one floor up_".
- The elevator should provide _grounding_ feedback to the user. For instance, it should respond "_Ok, going to the $X$-th floor_" after a user request to move to $X$.  
- The elevator should handle misunderstandings and uncertainties, e.g. by requesting the user to repeat, or asking the user to confirm if the intent is uncertain (say, when its confidence score is lower than 0.5). 
- The elevator should also allow the user to ask where the office of a given employee is located. For instance, the user could ask "_where is Erik Velldal's office?_", and the elevator would provide a response such as "_The office of Erik Velldal is on the 4th floor. Do you wish to go there?_".  We provide you with the office numbers of a small set of IFI employees in the `OFFICES` dictionary (see below).
- The elevator should also be able to inform the user about the current floor (such as replying to "_Which floor are we on?_" or "_Are we on the 5th floor?_"). 
- Finally, if the user asks the elevator to stop (or if the user says "_no_" after a grounding feedback "_Ok, going to floor $X$._"), the elevator should stop, and ask for clarification regarding the actual user intent. 

To implement this conversational behaviour, we will rely on a classical NLU-based approach in which we will recognise the user _intent_, and then determine a response based on the recognised intent(s). 

__Task 1.1__ (1 point): You first need to define a list of user intents that cover the kinds of user inputs you expect to observe in this talking elevator, such as `RequestMoveToFloor` or `Confirm`. This is a design question, and there is no obvious right or wrong answer. Define below the intents you want to cover, along with an explanation and a few examples of user inputs for each.


In [5]:
user_intents = [
    "Please go to the 1st floor.",
    "Floor 5, please.",
    "I want to go to floor 7.",
    "Go one floor up.",
    "Take me to the 2nd floor.",
    "What floor is this?",
    "Can you tell me where Anna's office is?",
    "I need to find Lars' office.",
    "Did you mean the 5th floor?",
    "Are you asking to go to the 7th floor?",
    "Stop!",
    "No, I changed my mind.",
    "Wait, I don’t want to go to that floor.",
    "I’m not sure what you meant. Can you say that again?"
]

__Task 1.2__ (1 points): We wish to build a classifier any user input to a probability distribution over those intents, and start by creating a small, synthetic training set. Make a list of about 100 user utterances, each labelled with an intent defined above. You can "make up" those utterances yourself, or ask someone else to come with alternative formulations if you lack inspiration.

In [6]:
labelled_utterances = [
    ("Please go to the 3rd floor.", "RequestMoveToFloor"),
    ("Floor 7, please.", "RequestMoveToFloor"),
    ("I want to go to the 5th floor.", "RequestMoveToFloor"),
    ("Can you take me to the 2nd floor?", "RequestMoveToFloor"),
    ("Go one floor up.", "RequestMoveToFloor"),
    ("Take me to floor 8.", "RequestMoveToFloor"),
    ("I need to go to the 1st floor.", "RequestMoveToFloor"),
    ("Go down to the 4th floor.", "RequestMoveToFloor"),
    ("I'd like to reach the 10th floor.", "RequestMoveToFloor"),
    ("Could you bring me to the 6th floor?", "RequestMoveToFloor"),
    ("Let's go to floor 9!", "RequestMoveToFloor"),
    ("One floor down, please.", "RequestMoveToFloor"),
    ("I want to go to the roof.", "RequestMoveToFloor"),
    ("Can you take me to the basement?", "RequestMoveToFloor"),
    ("Head to floor number 3.", "RequestMoveToFloor"),
    ("Up to the 2nd floor, please.", "RequestMoveToFloor"),
    ("Get me to the 5th floor, please.", "RequestMoveToFloor"),
    ("I need to go to the 4th floor now.", "RequestMoveToFloor"),
    ("Let's go to the ground floor.", "RequestMoveToFloor"),
    ("Please take me to floor 1.", "RequestMoveToFloor"),
    
    ("Ok, going to the 4th floor.", "GroundingFeedback"),
    ("Sure, I will take you to the 2nd floor now.", "GroundingFeedback"),
    ("Got it, heading to the 5th floor.", "GroundingFeedback"),
    ("Alright, going to floor 7.", "GroundingFeedback"),
    ("Okay, I’ll take you to the 3rd floor.", "GroundingFeedback"),
    
    ("Which floor are we on?", "RequestCurrentFloor"),
    ("Are we on the 3rd floor?", "RequestCurrentFloor"),
    ("What floor is this?", "RequestCurrentFloor"),
    ("Can you tell me what floor we're currently on?", "RequestCurrentFloor"),
    ("Please let me know the current floor.", "RequestCurrentFloor"),
    ("Am I on the 6th floor right now?", "RequestCurrentFloor"),
    
    ("Where is Erik Velldal's office?", "RequestEmployeeLocation"),
    ("Can you tell me where Anna's office is?", "RequestEmployeeLocation"),
    ("I need to find Lars' office.", "RequestEmployeeLocation"),
    ("Where can I find the office of Erik Velldal?", "RequestEmployeeLocation"),
    ("Could you point me to Anna's office?", "RequestEmployeeLocation"),
    ("I am looking for Lars' office, where is it?", "RequestEmployeeLocation"),
    ("Tell me where Erik Velldal works.", "RequestEmployeeLocation"),
    ("Where can I find the office of Anna?", "RequestEmployeeLocation"),
    
    ("Did you mean the 5th floor?", "Confirm"),
    ("Are you asking to go to the 7th floor?", "Confirm"),
    ("Is it correct that you want to go to the 2nd floor?", "Confirm"),
    ("Are you sure you want to go to floor 9?", "Confirm"),
    ("Did you say the 10th floor?", "Confirm"),
    
    ("Stop!", "RequestStop"),
    ("No, I changed my mind.", "RequestStop"),
    ("Wait, I don’t want to go to that floor.", "RequestStop"),
    ("Please stop here.", "RequestStop"),
    ("I want to get off now!", "RequestStop"),

    ("I didn't catch that, could you repeat?", "HandleMisunderstanding"),
    ("I’m not sure what you meant. Can you say that again?", "HandleMisunderstanding"),
    ("Can you clarify that?", "HandleMisunderstanding"),
    ("I don't understand, please repeat.", "HandleMisunderstanding"),
    ("What did you say? I didn't hear you.", "HandleMisunderstanding"),
    
    # Additional utterances for each intent
    ("Go to the 8th floor now.", "RequestMoveToFloor"),
    ("Can we go to the 4th floor?", "RequestMoveToFloor"),
    ("Elevator, go up one floor.", "RequestMoveToFloor"),
    
    ("Understood, heading to the 9th floor.", "GroundingFeedback"),
    ("Okay, we will go to floor 1.", "GroundingFeedback"),
    
    ("What floor are we on right now?", "RequestCurrentFloor"),
    ("Could you tell me the current level?", "RequestCurrentFloor"),
    
    ("Where's Lars' office located?", "RequestEmployeeLocation"),
    ("I need to know where Erik's office is.", "RequestEmployeeLocation"),
    
    ("You meant the 8th floor, right?", "Confirm"),
    ("So, you want to go to the 3rd floor?", "Confirm"),
    
    ("Please halt the elevator.", "RequestStop"),
    ("Stop going up!", "RequestStop"),
    
    ("I don't get what you said, please repeat it.", "HandleMisunderstanding"),
    ("Sorry, I didn’t catch that, could you say it again?", "HandleMisunderstanding"),

    # Additional examples for diverse utterances
    ("Bring me to the 5th floor.", "RequestMoveToFloor"),
    ("Let's stop here for a moment.", "RequestStop"),
    ("Can you stop the elevator, please?", "RequestStop"),
    ("Where is Anna's office on campus?", "RequestEmployeeLocation"),
    ("Confirm if I should go to the 3rd floor.", "Confirm"),
    ("Where are we now?", "RequestCurrentFloor"),
    ("Go down to the first floor, please.", "RequestMoveToFloor"),
    ("I want to go up to the 6th floor.", "RequestMoveToFloor"),
    ("Show me where Erik's office is.", "RequestEmployeeLocation"),
    ("Let me know which floor this is.", "RequestCurrentFloor"),
    ("I'm trying to find Lars' office.", "RequestEmployeeLocation"),
    ("Could you stop for a moment?", "RequestStop"),
    ("I need to go to the roof, please.", "RequestMoveToFloor"),
    ("I want to get off on the ground floor.", "RequestMoveToFloor"),
    ("What level are we on?", "RequestCurrentFloor"),
    ("I want to go to the 10th floor now.", "RequestMoveToFloor"),
    ("Could you take me down one floor?", "RequestMoveToFloor"),
    ("Stop at the next floor.", "RequestStop"),
    ("I need to confirm my floor.", "Confirm"),
    ("Do you mean floor 2?", "Confirm"),
    ("I want to go to floor 7, is that correct?", "Confirm"),
    ("Please repeat that, I didn’t hear you.", "HandleMisunderstanding"),
    ("What floor are we supposed to go to?", "HandleMisunderstanding"),
]

We will now train an intent classifier based on the labelled utterances you have defined. To do so, we will rely on the [SetFit](https://huggingface.co/docs/setfit/index) library, which allows one to easily train a text classification model from few examples by fine-tuning a sentence-transformer model (like the ones we used in oblig 2 and 3). Make sure that the `setfit` library is installed (`pip install setfit`).

Read the [Setfit quickstart guide](https://huggingface.co/docs/setfit/quickstart) to find out how to use the library.

__Task 1.3__ (2 points): Implement the `__init__`, `train` and `get_intent_distrib` methods of the `IntentClassifier` class below. The classifier should rely on a `Setfit` model trained on the labelled utterances you have already defined. 

In [12]:
import setfit, datasets
from typing import List, Tuple, Dict

class IntentClassifier:

    def __init__(self, model_name="sentence-transformers/paraphrase-mpnet-base-v2"):
        """Initialises the setfit model that will be used for the intent recognition"""
        self.model = setfit.SetFitModel.from_pretrained(model_name)
    
    def train(self, labelled_utterances: List[Tuple[str,str]]):
        """Trains the setfit model on the labelled utterances"""
        
        train_data = datasets.Dataset.from_list([{"text":utt, "label":label} 
                                                    for utt,label in labelled_utterances])
        self.model.fit(train_data)
    
    def get_intent_distrib(self, utterance:str):
        """Applies the trained model on a new utterance. The method should return a
        dictionary that maps each possible intent category to a probability."""
        
        probabilities = self.model.predict([utterance], return_proba=True)
        intent_distribution = {label: prob for label, prob in zip(self.model.config.id2label, probabilities[0])}
        
        return intent_distribution

In [18]:
import setfit, datasets
from typing import List, Tuple, Dict

class IntentClassifier:
    def __init__(self, model_name="sentence-transformers/paraphrase-mpnet-base-v2"):
        """Initialises the setfit model that will be used for the intent recognition"""
        self.model = setfit.SetFitModel.from_pretrained(model_name)
    
    def train(self, labelled_utterances: List[Tuple[str, str]], num_epochs: int = 3):
        """Trains the setfit model on the labelled utterances"""
        
        # Prepare data
        train_data = datasets.Dataset.from_list([{"text": utt, "label": label} 
                                                 for utt, label in labelled_utterances])
        
        # Extract texts and labels separately
        x_train = train_data['text']
        y_train = train_data['label']
        
        # Train the model
        self.model.fit(x_train, y_train, num_epochs=num_epochs)
    
    def get_intent_distrib(self, utterance: str) -> Dict[str, float]:
        """Applies the trained model on a new utterance. The method should return a
        dictionary that maps each possible intent category to a probability."""
        
        probabilities = self.model.predict([utterance], return_proba=True)
        intent_distribution = {label: prob for label, prob in zip(self.model.config.id2label, probabilities[0])}
        
        return intent_distribution

In [19]:
classifier = IntentClassifier()
classifier.train(labelled_utterances)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Since we don't have any test data, we cannot really conduct an evaluation of the classification performance, but this step would be of course strongly adviced when developing a real system. 

### Slot filling

In addition to the intents themselves, we also wish to detect some slots, such as floor numbers or person names. For this step, we will not use a data-driven model, but rather rely on an old-fashioned, rule-based approach:
- For floor numbers, we will rely on string matching (with regular expressions or basic string search) that detect patterns such as "X floor" (where X is [first,second, third, fourth, fifth, sixth, seventh, eighth, ninth, tenth]) or "floor X" (where X is between 1 and 10).
- For person names, we have a predefined list of person names to detect (employees at IFI), and we should simply search for their occurrence in the user input. The simplest implementation is to just for look for exact occurrences. However, since speech recognition will often struggle to recognize foreign person names, an even better approach would be to search for names that are phonetically close (you can use the `jellyfish` library for this).

The results of the slot filling should be a dictionary mapping slot names to a canonical form of the slot value. For instance, if the utterance contains the expression "ninth floor", the resulting slot dictionary should be `{"floor_number":9}`. Similarly, the `employee_name` slot should be a name present in `OFFICES` dictionary. 

__Task 1.4__ (2 points): Implement the method `fill_slots` that will detect the occurrence of those slots in the user input.<br>
(+ 1 bonus point if you implement a fuzzy matching strategy to find person names that are phonetically close)

In [22]:
import re
import jellyfish
from typing import Dict, Optional

# Floor numbers for a subset of the IFI employees
OFFICES = {
    'Adín Ramírez Rivera': 4, 'Andreas Austeng': 4, 'Anne H Schistad Solberg': 4, 
    'Arild Torolv Søetorp Waaler': 9, 'Audun Jøsang': 9, 'Birthe Soppe': 4, 
    'Carsten Griwodz': 4, 'Dag Sjøberg': 9, 'Dag Trygve Eckhoff Wisland': 5, 
    'Einar Broch Johnsen': 8, 'Eric Bartley Jul': 10, 'Erik Velldal': 4, 
    'Henrik Skaug Sætra': 7, 'Ingrid Chieh Yu': 8, 'Jørn Anders Braa': 6, 
    'Kristin Bråthen': 4, 'Kyrre Glette': 4, 'Lars Groth': 6, 'Lilja Øvrelid': 4, 
    'Maja Van Der Velden': 7, 'Martin Giese': 9, 'Michael Welzl': 5, 
    'Miria Grisot': 6, 'Nils Gruschka': 9, 'Olaf Owe': 9, 
    'Ole Christian Lingjærde': 4, 'Ole Hanseth': 6, 'Paulo Ferreira': 10, 
    'Philipp Dominik Häfliger': 5, 'Philipp Häfliger': 5, 'Roman Vitenberg': 4, 
    'Silvia Lizeth Tapia Tarifa': 8, 'Stephan Oepen': 4, 'Sundeep Sahay': 6, 
    'Thomas Peter Plagemann': 4, 'Tone Bratteteig': 7, 'Torbjørn Rognes': 8, 
    'Truls Erikson': 6, 'Viktoria Stray': 10, 'Yngvar Berg': 5, 
    'Yves Scherrer': 4, 'Özgü Mira Alay-Erduran': 4
}

def fill_slots(user_input: str) -> Dict[str, Optional[str]]:
    """Extracts the set of slots detected in the user inputs. More precisely, the method
    should detect both floor numbers and person names, and return a dictionary mapping slot 
    names (in this case either `floor_number` or `employee_name`) to its corresponding
    value, in canonical form (integer for the floor number, string for the employee name)"""
    
    slots = {}

    floor_patterns = [
        r"(?i)(first|second|third|fourth|fifth|sixth|seventh|eighth|ninth|tenth) floor",  # Words
        r"(?i)floor (1|2|3|4|5|6|7|8|9|10)"  # Numbers
    ]

    for pattern in floor_patterns:
        match = re.search(pattern, user_input)
        if match:
            # Handle the word-based pattern
            if match.group(0).startswith(('first', 'second', 'third', 'fourth', 'fifth', 
                                            'sixth', 'seventh', 'eighth', 'ninth', 'tenth')):
                floor_number = ['first', 'second', 'third', 'fourth', 'fifth', 
                                'sixth', 'seventh', 'eighth', 'ninth', 'tenth'].index(match.group(0).split()[0]) + 1
            else:
                floor_number = int(match.group(0).split()[1])
            slots['floor_number'] = floor_number
            break

    for name in OFFICES.keys():
        if name.lower() in user_input.lower():
            slots['employee_name'] = name
            break

    if 'employee_name' not in slots:
        for name in OFFICES.keys():
            if jellyfish.jaro_winkler_similarity(name.lower(), user_input.lower()) > 0.8:
                slots['employee_name'] = name
                break

    return slots

### Response selection

The next step is to implement the response selection mechanism. The response will depend on various factors:
- the inferred user intents from the user utterance
- the detected slot values in the user utterance (if any)
- the current floor
- the list of next floor stops that are yet to be reached
- the dialogue history (as a list of dialogue turns).

The response may consist of verbal responses (enacted by calls to `_say_to_user`) but also physical actions, represented by calls to either `move_to_floor` or `stop`. 

__Task 1.5__ (3 points): Implement the method `_respond`, which is responsible for selecting and executing those responses. The responses should satisfy the aforementioned conversational criteria (provide grounding feedback, use confirmations and clarification requests etc.). This method will consist in practice of many _if...then...else_ blocks. 

In [34]:
def _respond(self, intent_distrib: Dict[str, float], slots: Dict[str, str]):
        """Given a probability distribution over possible intents, an a (possibly empty) list
        of detected slots in the user input, decide how to react. The method should lead
        to calls to both physical actions (move_to_floor, stop) and dialogue responses 
        (via _say_to_user)."""
        
        intent = max(intent_distrib, key=intent_distrib.get, default=None)
        if intent == 'move_to_floor':
            if 'floor_number' in slots:
                target_floor = slots['floor_number']
                if target_floor == self.current_floor:
                    self._say_to_user("You are already on this floor.")
                elif target_floor in self.next_floors:
                    self.move_to_floor(target_floor)
                    self.stop()
                    self._say_to_user(f"Arrived at floor {target_floor}.")
                    self.dialogue_history.append(f"Moved to floor {target_floor}.")
                else:
                    self._say_to_user(f"I'm sorry, but floor {target_floor} is not on the list of next stops.")
            else:
                self._say_to_user("Which floor would you like to go to?")

        elif intent == 'ask_next_floors':
            if self.next_floors:
                floors_str = ', '.join(str(floor) for floor in self.next_floors)
                self._say_to_user(f"The next stops are: {floors_str}.")
            else:
                self._say_to_user("There are no more stops scheduled.")

        elif intent == 'clarification_request':
            self._say_to_user("Could you please clarify your request?")
        
        else:
            self._say_to_user("I didn't quite understand that. Can you please repeat?")

        # Record dialogue history
        self.dialogue_history.append(f"User said: {slots}")

setattr(TalkingElevator, "_respond", _respond)

### Putting it all together

The last step is to implement the `process_input` method in the `TalkingElevator` class. The method should rely on the intent recognition, slot filling and response selection mechanism (which you have implemented in the previous steps) to react to a given user input.

**Task 1.6** (1 point): Implement the `process_input` method:

In [35]:
def process_input(self, user_input: str, conf_score: float = 1.0):
    """Processes the (transcribed) user input, and respond appropriately 
    (through a verbal response and possibly also an action, such as moving floors).
    The method should rely on the intent classifier, slot-filling function, and
    response selection function."""
    
    self._add_to_dialogue_history(user_input, speaker="user", conf_score=conf_score)
    intent_distrib = self.intent_recognition(user_input)
    slots = self.fill_slots(user_input)
    self._respond(intent_distrib, slots)
    
setattr(TalkingElevator, "process_input", process_input)

We are now ready to test our talking elevator: 


In [ ]:
elevator = TalkingElevator()

Your talking elevator will mostly likely not function properly right from the start. Identify what works and what doesn't and correct the code you have developed in Tasks 1.1 - 1.6 until your system meets the specifications we have outlined. 

## Part 2 : Machine translation

In this part, we evaluate a pre-trained machine translation model on data from the Lord of the Rings movies and fine-tune it to improve the translation quality.

### Data

We provide you with two files, `lotr.detok.de` and `lotr.detok.en`, containing German and English movie subtitles. These two files constitute a so-called _parallel corpus_, i.e. each sentence/line in German corresponds to a sentence/line in English. The two files have the same number of lines and the German sentence on line $i$ corresponds to the English sentence on line $i$. The subtitles are extracted from the [OpenSubtitles-2018](https://opus.nlpl.eu/OpenSubtitles/corpus/version/OpenSubtitles) corpus.

Here are the first ten lines of the two files:

<style scoped>
table {
  font-size: 12px;
}
</style>
| Nb  | German (`lotr.detok.de`)         | English (`lotr.detok.en`)      |
|---|----------------------------------|--------------------------------|
| 1 | Die Welt ist im Wandel. | The world is changed.   |
| 2 | Ich spüre es im Wasser. | I feel it in the water. |
| 3 | Ich spüre es in der Erde. | I feel it in the earth. |
| 4 | Ich rieche es in der Luft. | I smell it in the air. |
| 5 | Vieles, was einst war, ist verloren, da niemand mehr lebt, der sich erinnert. | Much that once was is lost. For none now live who remember it. |
| 6 | Es begann mit dem Schmieden der Großen Ringe. | It began with the forging of the Great Rings. |
| 7 | 3 wurden den Elben gegeben, den unsterblichen, weisesten und reinsten aller Wesen. | Three were given to the Elves: Immortal, wisest and fairest of all beings. |
| 8 | 7 den Zwergenherrschern, großen Bergleuten und Handwerkern in ihren Hallen aus Stein. | Seven to the Dwarf-lords: Great miners and craftsmen of the mountain halls. |
| 9 | Und 9... 9 Ringe wurden den Menschen geschenkt, die vor allem anderen nach Macht streben. | And nine nine rings were gifted to the race of Men who, above all else, desire power. |
| 10 | Denn diese Ringe bargen die Kraft und den Willen, jedes Volk zu leiten. | For within these rings was bound the strength and will to govern each race. |


### Getting started

We will a pretrained machine translation model for German-to-English translation. The model is available on the HuggingFace model hub and can be used with the `transformers` library.

Let us first make sure that all required modules are installed:

In [ ]:
# %pip install torch transformers accelerate evaluate sacrebleu sacremoses sentencepiece unbabel-comet

The bilingual model is called [`opus-mt-de-en`](https://huggingface.co/Helsinki-NLP/opus-mt-de-en) and has been trained by the Helsinki-NLP group. Like (almost) all HuggingFace models, it consists of a _tokenizer_ and the _sequence-to-sequence model_ properly speaking. We need to load both separately:

In [1]:
import transformers

tokenizer = transformers.AutoTokenizer.from_pretrained("helsinki-nlp/opus-mt-de-en")
translator = transformers.AutoModelForSeq2SeqLM.from_pretrained("helsinki-nlp/opus-mt-de-en")

# Change "cuda" to "cpu" if you're running on a machine without GPU
device = "cpu"
translator = translator.to(device)

/fp/projects01/ec30/software/easybuild/software/nlpl-pytorch/2.1.2-foss-2022b-cuda-12.0.0-Python-3.10.8/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


The `transformers` library will automatically download the models from the HuggingFace hub the first time you run this cell, so it may take a bit longer.

Let's take the first two German sentences, tokenize them, and translate them to English:

In [2]:
tokens = tokenizer(["Die Welt ist im Wandel.", "Ich spüre es im Wasser."], return_tensors="pt", padding=True)
print(tokens)

{'input_ids': tensor([[   55,   401,    29,    49,  9012,     3,     0, 58100, 58100],
        [  105,  2768,  1691,    18,    65,    49,   672,     3,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [3]:
outputs = translator.generate(**tokens.to(device), max_new_tokens=50)
print(outputs)

tensor([[58100,    36,   360,    19,  7315,     3,     0, 58100, 58100, 58100],
        [58100,    38,    85,  1595,    56,     5,     4,   616,     3,     0]])


__Task 2.1__ (1 point):
- What do the numbers in the `input_ids` represent?
    - The numbers in the input_ids tensor represent the tokenized input sentences. Each unique word or subword from the sentences is mapped to a specific integer ID according to the tokenizer's vocabulary. For example in the first output, the first row corresponds to the tokenized sentence "Die Welt ist im Wandel."

- What is the effect of `padding=True`? How would the data look like if padding was disabled?
    - With `padding=True` we ensures that all sequences in the batch are of the same length by adding padding tokens (usually represented by 0 or another designated ID) to shorter sequences. This is crucial for batch processing, as many models require fixed-size inputs. Here, the input_ids would not include the padding tokens, and the sequences would have varying lengths.

- What does `max_new_tokens` do? Why do you think it is important to set this parameter?
    - The `max_new_tokens` parameter specifies the maximum number of new tokens the model can generate during the translation process. It's important basically for controlling the length of the output and ensuring it does not exceed a certain limit.

We can get actual words by running the output through the `batch_decode` function of the tokenizer:

In [4]:
translations = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(translations)

['The world is changing.', 'I can feel it in the water.']


__Note:__ We assume that you will run the translations from German to English. If you would like to work on the opposite translation direction (and feel comfortable evaluating the German output), you are welcome to do so. The corresponding bilingual model is called `opus-mt-en-de`.

In [5]:
import transformers

tokenizer = transformers.AutoTokenizer.from_pretrained("helsinki-nlp/opus-mt-en-de")
translator = transformers.AutoModelForSeq2SeqLM.from_pretrained("helsinki-nlp/opus-mt-en-de")

device = "cpu"
translator = translator.to(device)

tokens = tokenizer(["The world is changing.", "I can feel it in the water."], return_tensors="pt", padding=True)
print(tokens)

{'input_ids': tensor([[   36,   360,    19,  7315,     3,     0, 58100, 58100, 58100],
        [   38,    85,  1595,    56,     5,     4,   616,     3,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [6]:
outputs = translator.generate(**tokens.to(device), max_new_tokens=50)
print(outputs)

tensor([[58100,    55,   401,  6190,    63,     3,     0, 58100, 58100, 58100],
        [58100,   105,  2768,  1691,    18,    65,    49,   672,     3,     0]])


In [7]:
translations = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(translations)

['Die Welt verändert sich.', 'Ich spüre es im Wasser.']


### Data splitting

Before we move on, we need to split our data. We will evaluate different models and for that we'll need test data. We will also fine-tune a model, and for that we'll need training data. The entire Lord of the Rings dataset has 9640 lines.

__Task 2.2__ (1 point): Split the dataset in such a way that the **last** 1000 lines are used for testing and the remaining lines (8640) for training. Save the data under the following filenames: `lotr.train.de, lotr.train.en, lotr.test.de, lotr.test.en`. You can use Python code or other tools to perform the splitting.

In [8]:
with open('lotr.detok.de', 'r', encoding='utf-8') as f:
    german_lines = f.readlines()

with open('lotr.detok.en', 'r', encoding='utf-8') as f:
    english_lines = f.readlines()

print(f'Total lines: {len(german_lines)}')

Total lines: 9640


In [9]:
train_german = german_lines[:-1000]
test_german = german_lines[-1000:]

train_english = english_lines[:-1000]
test_english = english_lines[-1000:]

# Saving
with open('lotr.train.de', 'w', encoding='utf-8') as f:
    f.writelines(train_german)

with open('lotr.train.en', 'w', encoding='utf-8') as f:
    f.writelines(train_english)

with open('lotr.test.de', 'w', encoding='utf-8') as f:
    f.writelines(test_german)

with open('lotr.test.en', 'w', encoding='utf-8') as f:
    f.writelines(test_english)

__Task 2.3__ (1 point): What are potential risks and drawbacks of splitting the dataset in this way? 


Splitting a dataset it's not a greate idea because there are a lot of potential issues. One major concern is temporal bias, especially if the dataset is ordered chronologically; the last lines may not represent the same distribution as the earlier ones, leading to a model that performs poorly on the test data due to context or style differences. This method can cause data leakage if dependencies exist between the training and testing sets, inflating performance metrics and misguiding evaluations. This split also lacks randomness, which could lead to unbalanced representations of the data in both sets, increasing the risk of overfitting. Maybe we can use stratified sampling or random shuffling.

Now we are ready to translate the test set with our model.

__Task 2.4__ (2 points): Create a function that loads the entire `lotr.test.de` file, translates each line with the `opus-mt-de-en` model and writes its output to a new file, one sentence per line.

The easiest way to do this is to just load the entire test file into a list, tokenize and translate it, but the test set may be too large to fit on GPU memory, or it might be inefficient and slow if you use a CPU. A better alternative is to split the data into batches of 50-100 sentences and send each batch separately to the translator.

In [10]:
import transformers
import torch
from tqdm import tqdm

def translate(input_file, translation_file, tokenizer, translator, batch_size=100):
    """Translate an input file line by line using the loaded tokenizer and translator,
    and write the translations to output_file."""
    
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    translated_lines = []
    total_batches = len(lines) // batch_size + (1 if len(lines) % batch_size != 0 else 0)

    # tqdm for progress bar
    for i in tqdm(range(0, len(lines), batch_size), total=total_batches, desc="Translating"):
        batch = lines[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        inputs = {key: value.to(translator.device) for key, value in inputs.items()}
        
        with torch.no_grad():  # Disable gradient calculation
            outputs = translator.generate(**inputs)

        translated_batch = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        translated_lines.extend(translated_batch)

    with open(translation_file, 'w', encoding='utf-8') as f:
        for line in translated_lines:
            f.write(line + '\n')

translate("lotr.test.de", "lotr.output_opus.en", tokenizer, translator)

Translating: 100%|██████████| 10/10 [24:08<00:00, 144.83s/it]


Before moving on, open the output file and check that the translations look ok. In particular, the file should contain the expected number of lines and output should be in the expected language (English or German, depending on the chosen direction).

__Task 2.5__ (1 point): Open both the output file and the reference translations (`lotr.test.en` if translating from German to English) and compare the first 20 lines. How would you rate the translations of the OPUS system on a scale from 1 (incomprehensible and/or completely different meaning) to 5 (grammatically correct and meaning fully preserved)? Justify your answer.

I'm giving a 3/5 because the OPUS translations for the first 20 lines of the German text generally convey the intended meaning, but they fall short of fully preserving the nuances and fluidity expected in high-quality translations. Many phrases are translated too literally, resulting in awkward or unnatural English that disrupts readability. While the grammar is mostly correct, certain sentences contain structural issues, leading to somewhat fragmented output that affects fluency.

### Evaluation

We can now evaluate the quality of our translations. In a first step, we perform _reference-based surface-level evaluation_  using the popular BLEU score. We can do that with the `sacrebleu` module. Below is a slightly reformatted example taken from the [SacreBLEU documentation](https://github.com/mjpost/sacrebleu/tree/master?tab=readme-ov-file#using-sacrebleu-from-python):

In [11]:
from sacrebleu.metrics import BLEU

reference = ['The dog bit the man.', 'It was not unexpected.', 'The man bit him first.']
hypothesis = ['The dog bit the man.', "It wasn't surprising.", 'The man had just bitten him.']

bleu_scorer = BLEU()
# BLEU can deal with multiple references per sentence, but here we only have one, so we just enclose it in another set of brackets:
score = bleu_scorer.corpus_score(hypothesis, [reference])
print(score)

BLEU = 45.07 70.6/42.9/36.4/37.5 (BP = 1.000 ratio = 1.000 hyp_len = 17 ref_len = 17)


__Task 2.6__ (1 point): Load both the system output and the reference of your test set and compute the corpus-level BLEU score. Also compute the corpus-level chrF score. Which of the scores is higher?

In [12]:
from sacrebleu.metrics import BLEU, CHRF

def evaluate_bleu_chrf(hypothesis_file, reference_file):
    bleu = BLEU()
    chrf = CHRF()
    
    with open(hypothesis_file, 'r', encoding='utf-8') as hyp_file:
        hypotheses = [line.strip() for line in hyp_file.readlines()]
    with open(reference_file, 'r', encoding='utf-8') as ref_file:
        references = [line.strip() for line in ref_file.readlines()]
    
    # scores
    bleu_score = bleu.corpus_score(hypotheses, [references]).score
    chrf_score = chrf.corpus_score(hypotheses, [references]).score
    print(f"Corpus-level BLEU score: {bleu_score:.2f}")
    print(f"Corpus-level chrF score: {chrf_score:.2f}")

# Example usage
evaluate_bleu_chrf("lotr.output_opus.en", "lotr.test.en")

Corpus-level BLEU score: 0.98
Corpus-level chrF score: 14.60


Besides string-based metrics, neural metrics have become increasingly popular lately, since they have been shown to correlate better with human judgements. The most popular neural metric is called COMET and it can be used with the HuggingFace `evaluate` package. The example below is from the [documentation](https://huggingface.co/spaces/evaluate-metric/comet/blob/main/README.md):

In [13]:
import evaluate

comet_metric = evaluate.load('comet')
src = ["Dem Feuer konnte Einhalt geboten werden", "Schulen und Kindergärten wurden eröffnet."]
hyp = ["The fire could be stopped", "Schools and kindergartens were open"]
ref = ["They were able to control the fire.", "Schools and kindergartens opened"]
comet_score = comet_metric.compute(predictions=hyp, references=ref, sources=src)
print(comet_score)

2024-10-31 23:28:06.134433: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-31 23:28:06.134508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-31 23:28:06.215823: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-31 23:28:06.288310: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-31 23:28:11.868374: W tensorflow/compiler/tf2

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/3.53k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

Encoder model frozen.
/fp/homes01/u01/ec-elisao/.local/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
/fp/homes01/u01/ec-elisao/.local/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /cluster/software/EL9/easybuild/software/jupyter-ser ...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


{'mean_score': 0.9051420390605927, 'scores': [0.8385582566261292, 0.9717258214950562]}


__Task 2.7__ (1 point): Adapt this code to evaluate the output of the OPUS model. Note that COMET also requires the source text.

In [14]:
def evaluate_comet(hypothesis_file, reference_file, source_file):

    with open(hypothesis_file, 'r', encoding='utf-8') as hyp_file:
        hypotheses = [line.strip() for line in hyp_file.readlines()]
    
    with open(reference_file, 'r', encoding='utf-8') as ref_file:
        references = [line.strip() for line in ref_file.readlines()]
    
    with open(source_file, 'r', encoding='utf-8') as src_file:
        sources = [line.strip() for line in src_file.readlines()]
    
    comet_score = comet_metric.compute(predictions=hypotheses, references=references, sources=sources)
    print(f"COMET Score: {comet_score}")

# Example usage
evaluate_comet("lotr.output_opus.en", "lotr.test.en", "lotr.test.de")

/fp/homes01/u01/ec-elisao/.local/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /cluster/software/EL9/easybuild/software/jupyter-ser ...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


COMET Score: {'mean_score': 0.7155076760649681, 'scores': [0.9114322662353516, 0.53718101978302, 0.7610780000686646, 0.7029255032539368, 0.9578304886817932, 0.8175437450408936, 0.4858451783657074, 0.8657262921333313, 0.6912862062454224, 0.4729771614074707, 0.6021385192871094, 0.5424554347991943, 0.3910665214061737, 0.6396649479866028, 0.7489166855812073, 0.8702802658081055, 0.8097953796386719, 0.9071325063705444, 0.6414850354194641, 0.7230848670005798, 0.47912877798080444, 0.6162793040275574, 0.7267097234725952, 0.7105870246887207, 0.8409775495529175, 0.7223947644233704, 0.7569881677627563, 0.32134532928466797, 0.5586471557617188, 0.7251235246658325, 0.5417190194129944, 0.5329115986824036, 0.8773753046989441, 0.43584537506103516, 0.27712786197662354, 0.3007590174674988, 0.5079101920127869, 0.5500727891921997, 0.44638609886169434, 0.4835907518863678, 0.8644341230392456, 0.7896643280982971, 0.5770344734191895, 0.8151125907897949, 0.9198241233825684, 0.6831172704696655, 0.4741860032081604

### Fine-tuning

Let us see now if we can further improve the translation quality. We still haven't used the training set after all...

Fine-tuning a translation model with the `transformers` library is a bit convoluted. You need the following ingredients:
- A `Seq2SeqTrainer` object, which defines the initial model and its tokenizer, the training data, and the configuration parameters (as a `Seq2SeqTrainingArguments` object). The training process starts with the `train()` method.
- A `Seq2SeqTrainingArguments` object, which contains the configuration parameters, such as the number of training epochs, the path for saving the fine-tuned model, the learning rate etc.
- A `DataCollatorForSeq2Seq` object that takes care of splitting the training data into batches of appropriate size.
- A `DatasetDict` object containing the tokenized training data. Typically, the untokenized data is loaded into a `DatasetDict` object, and the tokenization function is applied to everything inside this `DatasetDict` using the `map()` function.

__Task 2.8__ (1 point): The code in the box below shows a working example using the pretrained OPUS model, but is limited to two sentence pairs. Complete the code to load the entire training data.

In [ ]:
# from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
# from transformers import DataCollatorForSeq2Seq
# from datasets import Dataset, DatasetDict

# max_length = 100

# ds = Dataset.from_dict({
#     "src_text": ["Die Welt ist im Wandel.", "Ich spüre es im Wasser."],
#     "tgt_text": ["The world is changed.", "I feel it in the water."]
# })
# data = DatasetDict({"train": ds})

# def preprocess_function(examples):
#     model_inputs = tokenizer(examples["src_text"], text_target=examples["tgt_text"], max_length=max_length, truncation=True)
#     return model_inputs

# tokenized_datasets = data.map(preprocess_function, batched=True)
# print(tokenized_datasets)

# data_collator = DataCollatorForSeq2Seq(tokenizer, model=translator)

# args = Seq2SeqTrainingArguments(
#     output_dir="opus-mt-de-en-lotr",
#     evaluation_strategy="no",
#     save_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=32,
#     weight_decay=0.01,
#     save_total_limit=3,
#     num_train_epochs=3,
#     predict_with_generate=True
# )

# trainer = Seq2SeqTrainer(
#     translator,
#     args,
#     train_dataset=tokenized_datasets["train"],
#     data_collator=data_collator,
#     tokenizer=tokenizer
# )

# trainer.train()

In [15]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
from datasets import Dataset, DatasetDict

max_length = 100

with open("lotr.train.de", "r", encoding="utf-8") as src_file:
    src_text = [line.strip() for line in src_file.readlines()]

with open("lotr.train.en", "r", encoding="utf-8") as tgt_file:
    tgt_text = [line.strip() for line in tgt_file.readlines()]

ds = Dataset.from_dict({"src_text": src_text, "tgt_text": tgt_text})
data = DatasetDict({"train": ds})

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["src_text"], text_target=examples["tgt_text"],
        max_length=max_length, truncation=True
    )
    return model_inputs

# Tokenization
tokenized_datasets = data.map(preprocess_function, batched=True)
print(tokenized_datasets)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=translator)

args = Seq2SeqTrainingArguments(
    output_dir="opus-mt-de-en-lotr",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True
)

trainer = Seq2SeqTrainer(
    translator,
    args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

Map:   0%|          | 0/8640 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['src_text', 'tgt_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8640
    })
})


Step,Training Loss
500,2.690500


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[58100]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[58100]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=810, training_loss=2.516604953342014, metrics={'train_runtime': 1548.5583, 'train_samples_per_second': 16.738, 'train_steps_per_second': 0.523, 'total_flos': 248890269892608.0, 'train_loss': 2.516604953342014, 'epoch': 3.0})

The model was fine-tuned for three epochs, and you should have three checkpoints in the `opus-mt-de-en-lotr` directory.

__Task 2.9__ (1 point): Choose one of the checkpoints and use it to translate the test set. Evaluate the test set with BLEU, chrF and COMET. Note that locally saved model files (and tokenizers) can be loaded in the same way as models from the HuggingFace hub, e.g. with the following command: `transformers.AutoModelForSeq2SeqLM.from_pretrained("opus-mt-de-en-lotr/checkpoint-810")`

Did fine-tuning help? Did fine-tuning help? Have a look at the first rows of the files. Do you agree with the metrics?

In [16]:
import evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint_path = "opus-mt-de-en-lotr/checkpoint-540"  # checkpoint
tokenizer = transformers.AutoTokenizer.from_pretrained(checkpoint_path)
translator = transformers.AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path).to("cuda" if torch.cuda.is_available() else "cpu")

def translate_test_set(input_file, output_file, tokenizer, translator, batch_size=100):
    with open(input_file, 'r', encoding='utf-8') as src_file:
        src_lines = [line.strip() for line in src_file.readlines()]

    translated_lines = []
    for i in tqdm(range(0, len(src_lines), batch_size)):
        batch = src_lines[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(translator.device)
        with torch.no_grad():
            outputs = translator.generate(**inputs)
        translated_lines.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))

    with open(output_file, 'w', encoding='utf-8') as out_file:
        for line in translated_lines:
            out_file.write(line + "\n")

translate_test_set("lotr.test.de", "lotr.finetuned_output.en", tokenizer, translator)

bleu = evaluate.load('sacrebleu')
chrf = evaluate.load('chrf')
comet = evaluate.load('comet')

with open("lotr.finetuned_output.en", 'r', encoding='utf-8') as hyp_file:
    hypotheses = [line.strip() for line in hyp_file.readlines()]

with open("lotr.test.en", 'r', encoding='utf-8') as ref_file:
    references = [line.strip() for line in ref_file.readlines()]

with open("lotr.test.de", 'r', encoding='utf-8') as src_file:
    sources = [line.strip() for line in src_file.readlines()]

100%|██████████| 10/10 [03:01<00:00, 18.15s/it]


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint .cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/371e9839ca4e213dde891b066cf3080f75ec7e72/checkpoints/model.ckpt`
Encoder model frozen.
/fp/homes01/u01/ec-elisao/.local/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [17]:
# Compute and print BLEU, chrF, and COMET scores
flat_references = [ref for sublist in references for ref in sublist]

bleu_score = bleu.compute(predictions=hypotheses, references=[[ref] for ref in references])  # Wrap references
print(f"BLEU score: {bleu_score['score']:.2f}")

chrf_score = chrf.compute(predictions=hypotheses, references=[[ref] for ref in references])
print(f"chrF score: {chrf_score['score']:.2f}")

#comet_score = comet.compute(predictions=hypotheses, references=flat_references, sources=sources)
#print(f"COMET score: {comet_score['score']:.2f}")

BLEU score: 12.78
chrF score: 32.35


In [ ]:
# The evaluation of the fine-tuned translation model yielded a BLEU score of 12.78 and a chrF score of 32.35, suggesting modest performance.
#While some sentences convey a sense of meaning, many translations exhibit grammatical errors and awkward phrasing, indicating a lack of fluency and coherence.
#For example, phrases like "This stone could overall his" and "The Arkenstone lieves in this halls" demonstrate clear mistranslations and could confuse readers.